In [3]:
import datetime
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns
import cryptocompare as cc
import matplotlib.pyplot as plt

In [4]:
data = pd.read_csv('Data.csv')

In [5]:
data

,Date,Ranks,Crypto,Symbol,Market Cap,Closing Price,Supply,Unnamed: 7
0,05/05/13,1,Bitcoin,BTC,"$1,288,693,216.22",115.91,"11,118,050 BTC",NaN
1,05/05/13,2,Litecoin,LTC,"$62,298,217.32",3.59,"17,348,954 LTC",NaN
2,05/05/13,3,Namecoin,NMC,"$6,290,543.05",1.15,"5,465,350 NMC",NaN
3,05/05/13,4,Peercoin,PPC,"$5,718,446.46",0.30,"18,830,240 PPC",NaN
4,05/05/13,5,Feathercoin,FTC,"$2,017,436.23",0.31,"6,444,650 FTC",NaN
...,...,...,...,...,...,...,...,...
1035,05/12/21,6,Cardano,ADA,"$45,909,141,522.15",1.38,"33,313,246,915 ADA",NaN
1036,05/12/21,7,USD Coin,USDC,"$40,884,696,965.55",1.00,"40,937,321,254 USDC *",NaN
1037,05/12/21,8,XRP,XRP,"$38,062,181,528.71",0.81,"47,247,295,769 XRP *",NaN
1038,05/12/21,9,Polkadot,DOT,"$27,958,507,597.69",28.31,"987,579,315 DOT *",NaN


In [6]:
data.columns

Index(['Date', 'Ranks', 'Crypto', 'Symbol', 'Market Cap', 'Closing Price',
       'Supply', 'Unnamed: 7'],
      dtype='object')

In [7]:
data.drop(columns='Unnamed: 7', axis = 1, inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data['DCA'] = 100/data['Closing Price']

,Ranks,Crypto,Symbol,Market Cap,Closing Price,Supply,DCA
Date,,,,,,,
2013-05-05,1,Bitcoin,BTC,"$1,288,693,216.22",115.91,"11,118,050 BTC",0.862738
2013-05-05,2,Litecoin,LTC,"$62,298,217.32",3.59,"17,348,954 LTC",27.855153
2013-05-05,3,Namecoin,NMC,"$6,290,543.05",1.15,"5,465,350 NMC",86.956522
2013-05-05,4,Peercoin,PPC,"$5,718,446.46",0.30,"18,830,240 PPC",333.333333
2013-05-05,5,Feathercoin,FTC,"$2,017,436.23",0.31,"6,444,650 FTC",322.580645
...,...,...,...,...,...,...,...
2021-05-12,3,Binance Coin,BNB,"$93,038,857,104.01",557.78,"166,801,148 BNB *",0.179282
2021-05-12,4,Tether,USDT,"$75,159,692,180.87",1.00,"75,112,354,134 USDT *",100.000000
2021-05-12,5,Solana,SOL,"$59,983,315,118.44",196.17,"305,774,824 SOL *",0.509762


In [ ]:
cc.cryptocompare._set_api_key_parameter('key')

'&api_key=51eb55dad7b62928816f26f48bfbbc7901d39b24bca350e763ded7f50de125ba'

In [234]:
#Using a copy of the original data and getting rid of unnecessary columns.

testingData = data.drop(columns = {'Market Cap', 'Supply'}, axis = 1)
exitsvalue = 0
exitcount = 0

# Optional : Year filter
testingData = testingData.loc[testingData.index.year > 2018]
testingData = testingData.loc[testingData['Ranks'] < 3]
testingData['DCA'] = (1000/int(testingData['Ranks'].max()))/testingData['Closing Price']
# Data Wrangling: A lot of columns have 0 as closing price, which renders calculation useless. 
# So we try to pull this data back from the python library cryptocompare
if np.inf in testingData.loc[(testingData['Crypto'] == 'XRP')]['DCA'].to_list():
    testingData.loc[(testingData['Crypto'] == 'XRP') & (testingData['Closing Price'] == 0), 'Closing Price'] = 0.01
    testingData['DCA'] = 100/testingData['Closing Price']

# Resetting index 
testingData = testingData.reset_index()

# Optional: Crypto filter
# testingData = testingData.loc[testingData['Crypto'] == 'Bitcoin']


portfolio = pd.DataFrame(columns = {'Crypto', 'Closing Price', 'DCA', 'Symbol'})
investment = testingData['Date'].nunique()*1000
for i in testingData['Date'].unique():

    ## Date filter
    new = testingData.loc[testingData['Date'] == i]
    ## Creating an empty dataframe to creating a demo portolio

    temp = pd.DataFrame(columns = {'Crypto', 'Average', 'DCA', 'Count', 'Symbol'})

    for x in list(new['Crypto'].unique()):
        crypto = str(x)

        
        # print(f'Exit(s) in {pd.to_datetime(i).strftime("%B")}, {pd.to_datetime(i).year} : {exits}')
        ### For cryptos which were already in the portfolio
        if x in list(portfolio['Crypto'].unique()):
            count = int(portfolio.loc[portfolio['Crypto'] == x]['Count'])
            #### Buy is the average
            buy = (float(new.loc[new['Crypto'] == x]['Closing Price']) + 
                   float(portfolio.loc[portfolio['Crypto'] == x]['Average']*portfolio.loc[portfolio['Crypto'] == x]
                         ['Count']))/(count + 1)
            #### Totalling DCA of all months
            quantity = float(portfolio.loc[portfolio['Crypto'] == x]['DCA']) + float(new.loc[new['Crypto'] == x]['DCA'])
            #### Creating dictionary and appending to demo portfolio
            count = int(portfolio.loc[portfolio['Crypto'] == x]['Count'] + 1)
            symbol = str(portfolio.loc[portfolio['Crypto'] == x]['Symbol'].iloc[0])
            tempdata = {'Crypto': [crypto], 'Average' : [buy], 'DCA' : [quantity], 'Count' : [count], 'Symbol' : [symbol]}
            temp = temp.append(pd.DataFrame(data = tempdata), ignore_index=True)
        ### For new addition to our porfolio

        elif x not in list(portfolio['Crypto'].unique()):
            #### Buy is the average
            buy = float(new.loc[new['Crypto'] == x]['Closing Price'])
            #### Totalling DCA of all months
            quantity = float(new.loc[new['Crypto'] == x]['DCA'])

            symbol = str(new.loc[new['Crypto'] == x]['Symbol'].iloc[0])
            #### Creating dictionary and appending to demo portfolio
            tempdata = {'Crypto': [crypto], 'Average' : [buy], 'DCA' : [quantity], 'Count' : 1, 'Symbol' : [symbol]}
            temp = temp.append(pd.DataFrame(data = tempdata), ignore_index=True)

        
    ### For cryptos which exited our portfolio
    exits = [x for x in portfolio['Symbol'].unique().tolist() if x not in new['Symbol'].unique().tolist()]
    for exit in exits:
        if cc.get_historical_price(exit, 'USD', datetime.datetime(pd.to_datetime(i).year,pd.to_datetime(i).month,pd.to_datetime(i).day)) == None:
            print(f'Not found : {exit}')
        else:
            if cc.get_historical_price(exit, 'USD', datetime.datetime(pd.to_datetime(i).year,pd.to_datetime(i).month,pd.to_datetime(i).day))[exit]['USD'] != 0:
                value = float(portfolio.loc[portfolio['Symbol'] == exit]['DCA']) * cc.get_historical_price(exit, 'USD', 
                    datetime.datetime(pd.to_datetime(i).year,pd.to_datetime(i).month,pd.to_datetime(i).day))[exit]['USD']
                exitsvalue = exitsvalue + value
                exitcount = exitcount + int(portfolio.loc[portfolio['Symbol'] == exit]['Count'])
                del value
    ### Demo portfolio is our new portfolio:
    portfolio = temp

exitcount = exitcount*(1000/int(testingData['Ranks'].max()))
portfolio = portfolio.merge(testingData.loc[testingData['Date'] == '2021-05-12'][['Crypto', 'Closing Price']], 
                            on = 'Crypto')
portfolio['Final Value'] = portfolio['Closing Price'] * portfolio['DCA']
print('Investment : ' + str(investment))
print('Final value : ' + str(portfolio['Final Value'].sum() + exitsvalue))
print('Return : ' + str(((portfolio['Final Value'].sum()+exitsvalue)/(investment) *100)) + '%')
print(f'Value of booked P/L {exitsvalue}')
print(f'Booked P/L investment: {exitcount}')
portfolio['Return'] =  (portfolio['Final Value'] - (portfolio['Count']*1000/int(testingData['Ranks'].max()) /investment)) * 100
portfolio

Investment : 36000
Final value : 315907.0080779396
Return : 877.5194668831656%
Value of booked P/L 956.8338615953514
Booked P/L investment: 1000.0


,Average,Crypto,Count,Symbol,DCA,Closing Price,Final Value,Return
0,21525.931667,Bitcoin,36,BTC,1.713541,49368.85,84595.550957,8.45951e+06
1,1133.508529,Ethereum,34,ETH,54.868286,4198.32,230354.623259,2.30354e+07


8459505.095741985

'993.7365098573862%'

3999.9995999999996